<a href="https://colab.research.google.com/github/chasubeen/ML_lab/blob/main/2_Variational_Inference_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import copy
from collections import defaultdict
from scipy.special import digamma, polygamma, loggamma

# **1. Preprocessing**

## **1-1. Load data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data path

train_path = "/content/drive/MyDrive/ML_lab/2_LDA/data/pos_tag/train.txt"
test_path = "/content/drive/MyDrive/ML_lab/2_LDA/data/pos_tag/test.txt"

In [ ]:
def read_file(path):
  raw = open(path, 'r').read().split('\n') # 파일을 읽어와 줄마다 나누기
  data, doc = list(), list()

  for i, line in enumerate(raw):
    if line.strip(): # 빈 줄이 아니면
      word = line.split(' ')[0].lower() # 첫 번째 단어를 얻어온 후 소문자로 변환
      doc.append(word)
    else:
      data.append(doc) # EOD(End Of Document)
      doc = list()
  data.append(doc) # 맨 마지막 문서 추가

  return data

In [ ]:
# 데이터 불러오기
train_data = read_file(train_path)
test_data = read_file(test_path)

# 맨 마지막 문서는 제거
del train_data[-1], test_data[-1]

## **1-2. Preprocess Data**

In [ ]:
# some words that we do not want to consider
stopwords = [""," "]

In [ ]:
## remove numbers, special characters, etc.
# only consider words with alphabet

def only_alphabet(corpus,stopword_list = stopwords):
  data = list()
  for doc in corpus:
    temp = list()
    for word in doc:
      word = re.sub('[^a-z]', '', word) # non-alphabet 문자 제거(=> 정규표현식 활용)
      if word not in stopwords:
        temp.append(word)
    data.append(temp)

  return data

In [ ]:
train_data = only_alphabet(train_data)
test_data = only_alphabet(test_data)

In [ ]:
## count word occurrence in corpus

def count_vocab(corpus):
  vocab = defaultdict(int) # dictionary 형태로 word occurence를 기록
  for doc in corpus:
    for word in doc:
      vocab[word] += 1

  return vocab

In [ ]:
vocab_count = count_vocab(train_data)

In [ ]:
## 특정 빈도수 이상인 단어들만 고려
# sparse한 단어는 무시

def vocab_top(vocab,cnt):
  temp = defaultdict(int)
  for voca, count in vocab.items():
    if count > cnt:
      temp[voca] = count

  return temp

In [ ]:
vocab_count = vocab_top(vocab_count,cnt = 5)

In [ ]:
## filter out low-occurrence words

def filter_vocab(corpus,vocab):
  data = list()
  for doc in corpus:
    temp = list()
    for word in doc:
      if word in vocab.keys():
        temp.append(word)
    if temp:
      data.append(temp)

  return data

In [ ]:
train_data = filter_vocab(train_data,vocab_count)
test_data = filter_vocab(test_data,vocab_count)

In [ ]:
## construct voca-index-matching dictionary

def voca_index(vocab):
  vocab_to_index, index_to_vocab = dict(), dict()
  for i, voca in enumerate(vocab.keys()):
    vocab_to_index[voca] = i
    index_to_vocab[i] = voca

  return vocab_to_index, index_to_vocab

In [ ]:
v_t_i, i_t_v = voca_index(vocab_count)

In [ ]:
## convert corpus-with-words to corpus-with-index

def corpus_to_index(corpus,vocab_to_index):
  data = list()
  for doc in corpus:
    temp = list()
    for word in doc:
      temp.append(vocab_to_index[word])
    data.append(temp)

  return data

In [ ]:
train_data_idx = corpus_to_index(train_data,v_t_i)
test_data_idx = corpus_to_index(test_data,v_t_i)

# **2. Modeling**

In [ ]:
class LDA_VI:
  def __init__(self,docs, num_topic=10, vocab = None, alpha=1.,num_iter=100,lr=1e-3):
    self.docs = docs
    self.num_topic = num_topic
    self.vocab = vocab
    self.num_vocab = len(self.vocab)
    self.num_docs = len(self.docs)

    # Initialize alpha and beta
    self.alpha = np.ones(self.num_topic)*(1/self.num_topic)
    self.beta = np.ones((self.num_topic, self.num_vocab))*(1/self.num_vocab)
    # beta 초기값 업데이트
    for i in range(self.num_topic):
      for v in range(self.num_vocab):
        self.beta[i][v] += np.random.rand(1) * 0.01
    # beta normailize 진행
    for i in range(self.num_topic):
      self.beta[i] /= np.sum(self.beta[i])

    # Initialize gamma and phi
    self.gamma = np.ones((self.num_docs,self.num_topic))*(1/self.num_topic)
    self.phi = np.ones((self.num_docs, self.num_topic,self.num_vocab))*(1/self.num_vocab)

    # Initialize z (topic assignments for words in documents)
    self.z = list()
    for doc in self.docs:
      temp = list()
      for word in doc:
        temp.append(None)
      self.z.append(temp)

    # Initialize parameters for variational inference
    self.num_iter = num_iter
    self.lr = lr

  def update_gamma(self):
    ## Update gamma (document-topic distributions)
    # d: document의 index
    for d in range(self.num_docs):
      for k in range(self.num_topic):
        self.gamma[d, k] = self.alpha[k] + np.sum(self.phi[d, k])

  def update_phi(self):
    ## Update phi (topic-word distributions)
    for d in range(self.num_docs):
      for n in range(len(self.docs[d])):
        for k in range(self.num_topic):
          self.phi[d, k, n] = self.beta[k, self.docs[d][n]] * np.exp(digamma(self.gamma[d, k]))
        # phi 정규화
        self.phi[d, :, n] /= np.sum(self.phi[d, :, n])

  def update_beta(self):
    ## Update beta(topic-word prior)
    for k in range(self.num_topic):
      for v in range(self.num_vocab):
        for d in range(self.num_docs):
          for n in range(len(self.docs[d])):
            if self.docs[d][n] == v:
              self.beta[k, v] += self.phi[d, k, n]
      # beta 정규화
      self.beta[k] /= np.sum(self.beta[k])

  def update_alpha(self):
    ## Update alpha(document-topic prior)
    M = self.num_docs
    g = np.zeros(self.num_topic) # gradient
    H = np.zeros((self.num_topic, self.num_topic)) # Hessian matrix

    for i in range(self.num_topic):
      g[i] = M * ( digamma(np.sum(self.alpha)) - digamma(self.alpha[i]) )
      for d in range(M):
        g[i] += ( digamma(self.gamma[d, i]) - digamma(np.sum(self.gamma[d, :])) )
      for j in range(self.num_topic):
        H[i, j] = 0
        if i == j:
          H[i, j] = -M * polygamma(1, self.alpha[i])


    # alpha 업데이트(=> Newton-Raphson)
    deltaAlpha = np.dot(np.linalg.inv(H), g)

    for k in range(self.num_topic):
      self.alpha[k] -= deltaAlpha[k]

  def e_step(self):
    ## Perform E-step of the variational inference
    self.update_phi()
    self.update_gamma()

  def m_step(self):
    ## Perform M-step of the variational inference
    self.update_beta()
    self.update_alpha()

  def compute_elbo(self):
    ## Compute Evidence Lower Bound (ELBO)

    # 필요한 파라미터 정의
    elbo = 0
    M = self.num_docs
    k = self.num_topic

    # 1번 수식
    for d in range(M): # document
      for n in range(len(self.docs[d])):
        j = self.docs[d][n] # word idx
        for i in range(k): # topic
          elbo += self.phi[d, i, n] * np.log(self.beta[i, j])

    # 2번 수식
    for d in range(M):
      for n in range(len(self.docs[d])):
        for i in range(k):
          elbo += self.phi[d, i, n] * (digamma(self.gamma[d, i]) - digamma(np.sum(self.gamma[d, :])))

    # 3번 수식
    elbo += M * (loggamma(np.sum(self.alpha)) - np.sum(loggamma(self.alpha)))

    for d in range(M):
      for i in range(k):
        elbo += (self.alpha[i] - 1) * (digamma(self.gamma[d, i]) - digamma(np.sum(self.gamma[d, :])))

    # 4번 수식
    for d in range(M):
      for i in range(k):
        elbo -= (self.gamma[d, i] - 1) * (digamma(self.gamma[d, i]) - digamma(np.sum(self.gamma[d, :])))

    for d in range(M):
      elbo -= loggamma(np.sum(self.gamma[d, :])) - np.sum(loggamma(self.gamma[d, :]))

    # 5번 수식
    for d in range(M):
      for n in range(len(self.docs[d])):
        for i in range(k):
          elbo -= self.phi[d, i, n] * np.log(self.phi[d, i, n])

    return elbo

  def compute_perplexity(self, elbo):
    # 분모
    N = sum(len(doc) for doc in self.docs)

    perplexity = np.exp(-elbo / N)
    return perplexity

  def run(self):
    ## Run the variational inference algorithm
    for iter in range(self.num_iter):
      print(f"=== Iteration: {iter} ===")
      self.e_step()
      self.m_step()
      print(f"Alpha: {self.alpha}")

      elbo = self.compute_elbo()
      print(f"ELBO: {elbo}")
      perplexity = self.compute_perplexity(elbo)
      print(f"Perplexity: {perplexity}")

    print("EM Algorithm 끝!")

# **3. Run!!**

In [ ]:
model = LDA_VI(test_data_idx,num_topic = 10, vocab = v_t_i, alpha = 1.,num_iter = 50,lr = 1e-3)
model.run()

<ipython-input-19-d1147a38c77d>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.beta[i][v] += np.random.rand(1) * 0.01


=== Iteration: 0 ===
Alpha: [0.17279333 0.17228359 0.17252091 0.17245722 0.172115   0.17272764
 0.17260403 0.1721599  0.17285073 0.17177484]
ELBO: -235056.953146378
Perplexity: 1099.281008685372
=== Iteration: 1 ===
Alpha: [0.28385037 0.28215863 0.28297113 0.28278656 0.28162609 0.28362013
 0.28324008 0.28179221 0.28403409 0.28056687]
ELBO: -228605.70247866696
Perplexity: 907.0767586957061
=== Iteration: 2 ===
Alpha: [0.43749975 0.43388356 0.43566916 0.43532114 0.43275744 0.43691895
 0.43617032 0.43311106 0.43782767 0.43064685]
ELBO: -223708.92860575503
Perplexity: 783.9543787921336
=== Iteration: 3 ===
Alpha: [0.62796885 0.62226129 0.62515137 0.62464099 0.62035521 0.62676281
 0.62566491 0.62081973 0.62828803 0.61732632]
ELBO: -220133.6393700775
Perplexity: 704.7491850373685
=== Iteration: 4 ===
Alpha: [0.84134336 0.83428366 0.83794163 0.83729579 0.83139349 0.83921965
 0.83788237 0.83169124 0.84135693 0.8282353 ]
ELBO: -217563.1940924874
Perplexity: 652.7979727520324
=== Iteration: 5 ==

In [ ]:
## ELBO 시각화



# **4. Inference**

In [ ]:
## analyze each doc's topic proportion

for d, gamma in enumerate(model.gamma):
  print(f"Document {d} topic distribution: {gamma / np.sum(gamma)}")

In [ ]:
## analyze topic-word proportion

for k, beta in enumerate(model.beta):
  top_words = np.argsort(beta)[-10:]
  print(f"Topic {k} top words: {[i_t_v[i] for i in top_words]}")